<!--NOTEBOOK_HEADER-->
*This notebook contains material from [cbe30338-2021](https://jckantor.github.io/cbe30338-2021);
content is available [on Github](https://github.com/jckantor/cbe30338-2021.git).*


<!--NAVIGATION-->
< [5.2 Linear Blending Problems](https://jckantor.github.io/cbe30338-2021/05.02-Linear-Blending-Problem.html) | [Contents](toc.html) | [Tag Index](tag_index.html) | [5.4 Gasoline Blending](https://jckantor.github.io/cbe30338-2021/05.04-Gasoline-Blending.html) ><p><a href="https://colab.research.google.com/github/jckantor/cbe30338-2021/blob/master/docs/05.03-Homework_5.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://jckantor.github.io/cbe30338-2021/05.03-Homework_5.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>

# 5.3 Homework Assignment 4

## Austin Booth

Link:

Homework assigment 4 is intended to develop your skills in creating linear programming models. The lecture notes (notebook 5.2) will provide with a starting point for the following exercises. The problem data has been changed to provide a more meaningful example for sensitivity analysis.


You have been placed in charge of a metals reuse operation. A collection of raw materials are available with key parameters given in the following table.

| material | % carbon (C) | % copper (Cu) | % manganese (Mn) | amount (kg) | cost (\$/kg) | type |
| :---: | :---: | :---: | :---: | :---: | :---: | :---
| A | 2.5 | 0.0 | 1.3 | 4000 | 1.20 | iron alloy
| B | 3.0 | 0.0 | 0.8 | 3000 | 1.50 | iron alloy
| C | 0.0 | 0.3 | 0.0 | 6000 | 0.90 | iron alloy
| D | 0.0 | 90.0 | 0.0 | 5000 | 1.30 | copper alloy
| E | 0.0 | 96.0 | 4.0 | 2000 | 1.45 | copper alloy
| F | 0.0 | 0.4 | 1.2 | 3000 | 1.20 | aluminum alloy
| G | 0.0 | 0.6 | 0.0 | 2500 | 1.00 | aluminum alloy

A customer has requested 5000 kg of mix satisfying these  specifications:

| Component | min % | max % |
| :---: | :---: | :---: |
| C | 2.0 | 3.0 |
| Cu | 0.4 | 0.6 |
| Mn | 1.2 | 1.65 |

For convenience, the raw material data has been organized as a nested dictionaries.

In [2]:
data = {
    "A": {"C": 2.5, "Cu": 0.0, "Mn": 1.3, "amount": 4000, "cost": 1.20},
    "B": {"C": 3.0, "Cu": 0.0, "Mn": 0.8, "amount": 3000, "cost": 1.50},
    "C": {"C": 0.0, "Cu": 0.3, "Mn": 0.0, "amount": 6000, "cost": 0.90},
    "D": {"C": 0.0, "Cu": 90.0, "Mn": 0.0, "amount": 5000, "cost": 1.30},
    "E": {"C": 0.0, "Cu": 96.0, "Mn": 4.0, "amount": 2000, "cost": 1.45},
    "F": {"C": 0.0, "Cu": 0.4, "Mn": 1.2, "amount": 3000, "cost": 1.20},
    "G": {"C": 0.0, "Cu": 0.6, "Mn": 0.0, "amount": 2500, "cost": 1.00},
}

## 5.3.1 Exercise 1. Getting Started




Considering just the constraint on carbon content, adapt the `brew_blend` function from notebook 5.2 to find a minimum cost blend raw materials that yield 5000 kg of product with a carbon content between 2.0 and 3.0%. For this first exercise you can ignore the bounds on the available amount of each raw material.

* change the name of the function to `metal_blend`.
* `metal_blend` should accept arguments including
    * `data`, 
    * the required mass of final product, and
    *  the acceptable range of carbon content specified as a pair of numbers of  `[lower_bound, upper_bound]`.
* `metal_blend` should return the minimum cost, and the amounts of each raw material to include in the blend.

Demonstrate use of `metal_blend` to compute the required blend. From the results of the calculation to report

* the cost of the blend
* the total mass of the blend
* the mass of each raw material used in the blend
* the composition of the blend with regard to the species carbon, copper and managanese.

You may find it convenient to write a function for this purpose that can be used in the following exercise.



### 5.3.1.1 Solution

In [33]:
import numpy as np
import cvxpy as cp

def metal_blend(mass, carbon, data):
    
    # create lower and upper bounds from carbon input
    lower_boundC = carbon[0]
    upper_boundC = carbon[1]
    
    # create set of components
    components = set(data.keys())
    
    # create variables
    x = {c: cp.Variable(nonneg=True, name=c) for c in components}
    
    # create objective function
    total_cost = sum(x[c]*data[c]['cost'] for c in components)
    
    # create list of constraints
    constraints = [
        mass == sum(x[c] for c in components),
        0 >= sum(x[c]*(data[c]['C'] - upper_boundC) for c in components),
        0 <= sum(x[c]*(data[c]['C'] - lower_boundC) for c in components)
    ]
    
    # create and solve problem
    problem = cp.Problem(cp.Minimize(total_cost), constraints)
    problem.solve()
    
    # return results
    # min cost and optimal blend
    min_cost = problem.value
    optimal_blend = {c: x[c].value for c in components}
    
    # composition (averages) of C, Cu, Mn
    avg_C = sum(x[c].value*data[c]['C'] for c in components) / mass 
    avg_Cu = sum(x[c].value*(data[c]['Cu']) for c in components) / mass
    avg_Mn = sum(x[c].value*(data[c]['Mn']) for c in components) / mass

    
    return min_cost, optimal_blend, avg_C, avg_Cu, avg_Mn
 
mass = 5000
carbon = [2.0, 3.0]
min_cost, optimal_blend, avg_C, avg_Cu, avg_Mn = metal_blend(mass, carbon, data)

print('With cost minimized and C between 2.0 and 3.0%:')
print(f"Mass: {mass} kg ")
print(f"Total cost: ${min_cost:5.2f}")
print(f"C composition: {avg_C:5.2f}%")
print(f"Cu composition: {avg_Cu:5.2f}%")
print(f"Mn composition: {avg_Mn:5.2f}%")
print()
for c in sorted(optimal_blend.keys()):
    print(f"{c}: {optimal_blend[c]:5.2f}", 'kg')

With cost minimized and C between 2.0 and 3.0%:
Mass: 5000 kg 
Total cost: $5700.00
C composition:  2.00%
Cu composition:  0.06%
Mn composition:  1.04%

A: 4000.00 kg
B:  0.00 kg
C: 1000.00 kg
D:  0.00 kg
E:  0.00 kg
F:  0.00 kg
G:  0.00 kg


## 5.3.2 Exercise 2. Incorporating Constraints

We'll continue this problem by incorporating all of the constraints. The constraiints include:

* Lower and upper bounds on the copper and manganese composition
* Limits on the amount of available raw material

The easiest way to proceed is to copy and paste `metal_blend` into a cell below, then add constraints one at a time until all have been incorporated.

What is the minimum price you would need to charge in $/kg to produce 5,000 kg of requested product?

In [52]:
import numpy as np
import cvxpy as cp

def metal_blend(mass, carbon, copper, manganese, data):
    
    # create lower and upper bounds from metal constraint inputs
    lower_boundC = carbon[0]
    upper_boundC = carbon[1]
    lower_boundCu = copper[0]
    upper_boundCu = copper[1]
    lower_boundMn = manganese[0]
    upper_boundMn = manganese[1]
    
    # create set of components
    components = set(data.keys())
    
    # create variables
    x = {c: cp.Variable(nonneg=True, name=c) for c in components}
    
    # create objective function
    total_cost = sum(x[c]*data[c]['cost'] for c in components)
    
    # create list of constraints
    constraints = [
        # mass
        mass == sum(x[c] for c in components),
        # carbon
        0 >= sum(x[c]*(data[c]['C'] - upper_boundC) for c in components),
        0 <= sum(x[c]*(data[c]['C'] - lower_boundC) for c in components),
        # copper
        0 >= sum(x[c]*(data[c]['Cu'] - upper_boundCu) for c in components),
        0 <= sum(x[c]*(data[c]['Cu'] - lower_boundCu) for c in components),
        # manganese
        0 >= sum(x[c]*(data[c]['Mn'] - upper_boundMn) for c in components),
        0 <= sum(x[c]*(data[c]['Mn'] - lower_boundMn) for c in components),
        # limits on each material
        0 >= x['A'] - data['A']['amount'],
        0 >= x['B'] - data['B']['amount'],
        0 >= x['C'] - data['C']['amount'],
        0 >= x['D'] - data['D']['amount'],
        0 >= x['E'] - data['E']['amount'],
        0 >= x['F'] - data['F']['amount'],
        0 >= x['G'] - data['G']['amount']


    ]
    
    # create and solve problem
    problem = cp.Problem(cp.Minimize(total_cost), constraints)
    problem.solve()
    
    # return results
    # min cost and optimal blend
    min_cost = problem.value
    optimal_blend = {c: x[c].value for c in components}
    
    # composition (averages) of C, Cu, Mn
    avg_C = sum(x[c].value*data[c]['C'] for c in components) / mass 
    avg_Cu = sum(x[c].value*(data[c]['Cu']) for c in components) / mass
    avg_Mn = sum(x[c].value*(data[c]['Mn']) for c in components) / mass

    
    return min_cost, optimal_blend, avg_C, avg_Cu, avg_Mn

# define constant constraint variables
mass = 5000
carbon = [2.0, 3.0]
copper = [0.4, 0.6]
manganese = [1.2, 1.65]

# execute function and print results
min_cost, optimal_blend, avg_C, avg_Cu, avg_Mn = metal_blend(mass, carbon, copper, manganese, data)

print('With cost minimized:')
print(f"Mass: {mass} kg ")
print(f"Total cost: ${min_cost:5.2f}")
print(f"C composition: {avg_C:5.2f}%")
print(f"Cu composition: {avg_Cu:5.2f}%")
print(f"Mn composition: {avg_Mn:5.2f}%")
print()
for c in sorted(optimal_blend.keys()):
    print(f"{c}: {optimal_blend[c]:5.2f}", 'kg')
    
# cost per kg    
print()
print(f'Cost per kg: ${(min_cost/mass):5.2f}/kg')

With cost minimized:
Mass: 5000 kg 
Total cost: $5887.57
C composition:  2.00%
Cu composition:  0.60%
Mn composition:  1.20%

A: 4000.00 kg
B:  0.00 kg
C: 397.76 kg
D:  0.00 kg
E: 27.61 kg
F: 574.62 kg
G:  0.00 kg

Cost per kg: $ 1.18/kg


## 5.3.3 Exercise 3. Sensitivity Analysis

One of the important reasons to create optimization models is to understand the value of the raw materials that are available to you, and how to adjust operations to accomodate changing requirements. The is commonly referred to as **sensitivity analysis**.

Using the functions you've created in above exercises, answer the following questions:

1. The customer is very pleased with your initial pricing for 5,000 kg of the desired product, and now wishes to increase the order to 6,000 kg. Does your unit cost ($/kg) go up? If so, explain why your unit cost has gone up.

2. Is there an upper limit on how much product your can provide to this customer? (Use trial and error. To the nearest 100 kg, find the maximum amount of product you can ship to the customer.) What is the unit cost now?

3. After some negotiation, you and your customer settle on an order of 6,500 kg. Now you wish to negotiate with your suppliers for more raw material. How much money to you save for 1 additional kg of raw material "A"? Based on this, what price would you be willing to pay your supplier for additional "A"?



### Part 1

In [53]:
# define constant constraint variables
mass = 6000
carbon = [2.0, 3.0]
copper = [0.4, 0.6]
manganese = [1.2, 1.65]

# execute function and print results
min_cost, optimal_blend, avg_C, avg_Cu, avg_Mn = metal_blend(mass, carbon, copper, manganese, data)

print('With cost minimized:')
print(f"Mass: {mass} kg ")
print(f"Total cost: ${min_cost:5.2f}")
print(f"C composition: {avg_C:5.2f}%")
print(f"Cu composition: {avg_Cu:5.2f}%")
print(f"Mn composition: {avg_Mn:5.2f}%")
print()
for c in sorted(optimal_blend.keys()):
    print(f"{c}: {optimal_blend[c]:5.2f}", 'kg')
    
# cost per kg    
print()
print(f'Cost per kg: ${(min_cost/mass):5.2f}/kg')

With cost minimized:
Mass: 6000 kg 
Total cost: $7352.14
C composition:  2.00%
Cu composition:  0.60%
Mn composition:  1.20%

A: 4000.00 kg
B: 666.67 kg
C: 186.41 kg
D:  0.00 kg
E: 32.27 kg
F: 1114.65 kg
G:  0.00 kg

Cost per kg: $ 1.23/kg


Yes, the unit cost has gone up. This is because some of component B now has to be used to contribute to the product's carbon content, since the maximum amount of component A is being used and is not enough to give the higher-quantity product 2% C. B must be added and is more expensive than A. Additionally, more of the expensive components F and E must be used to ensure that the entire product at the new quantity has a sufficient concentration of manganese.

### Part 2

In [79]:
# define constant constraint variables
mass = 6882
carbon = [2.0, 3.0]
copper = [0.4, 0.6]
manganese = [1.2, 1.65]

# execute function and print results
min_cost, optimal_blend, avg_C, avg_Cu, avg_Mn = metal_blend(mass, carbon, copper, manganese, data)

print('With cost minimized:')
print(f"Mass: {mass} kg ")
print(f"Total cost: ${min_cost:5.2f}")
print(f"C composition: {avg_C:5.2f}%")
print(f"Cu composition: {avg_Cu:5.2f}%")
print(f"Mn composition: {avg_Mn:5.2f}%")
print()
for c in sorted(optimal_blend.keys()):
    print(f"{c}: {optimal_blend[c]:5.2f}", 'kg')
    
# cost per kg    
print()
print(f'Cost per kg: ${(min_cost/mass):5.2f}/kg')

With cost minimized:
Mass: 6882 kg 
Total cost: $8643.89
C composition:  2.00%
Cu composition:  0.60%
Mn composition:  1.20%

A: 4000.00 kg
B: 1254.67 kg
C:  0.01 kg
D:  0.00 kg
E: 36.38 kg
F: 1590.94 kg
G:  0.00 kg

Cost per kg: $ 1.26/kg


The program errors out past this point. Thus, the maximum amount of product is about 6900 kg, with a cost per kg of $1.26/kg.

### Part 3

In [80]:
def metal_blend(mass, carbon, copper, manganese, data):
    
    # create lower and upper bounds from metal constraint inputs
    lower_boundC = carbon[0]
    upper_boundC = carbon[1]
    lower_boundCu = copper[0]
    upper_boundCu = copper[1]
    lower_boundMn = manganese[0]
    upper_boundMn = manganese[1]
    
    # create set of components
    components = set(data.keys())
    
    # create variables
    x = {c: cp.Variable(nonneg=True, name=c) for c in components}
    
    # create objective function
    total_cost = sum(x[c]*data[c]['cost'] for c in components)
    
    # create list of constraints
    constraints = [
        # mass
        mass == sum(x[c] for c in components),
        # carbon
        0 >= sum(x[c]*(data[c]['C'] - upper_boundC) for c in components),
        0 <= sum(x[c]*(data[c]['C'] - lower_boundC) for c in components),
        # copper
        0 >= sum(x[c]*(data[c]['Cu'] - upper_boundCu) for c in components),
        0 <= sum(x[c]*(data[c]['Cu'] - lower_boundCu) for c in components),
        # manganese
        0 >= sum(x[c]*(data[c]['Mn'] - upper_boundMn) for c in components),
        0 <= sum(x[c]*(data[c]['Mn'] - lower_boundMn) for c in components),
        # limits on each material
        0 >= x['A'] - data['A']['amount'],
        0 >= x['B'] - data['B']['amount'],
        0 >= x['C'] - data['C']['amount'],
        0 >= x['D'] - data['D']['amount'],
        0 >= x['E'] - data['E']['amount'],
        0 >= x['F'] - data['F']['amount'],
        0 >= x['G'] - data['G']['amount']
    ]
    
    # create and solve problem
    problem = cp.Problem(cp.Minimize(total_cost), constraints)
    problem.solve()
    
    # return results
    # min cost and optimal blend
    min_cost = problem.value
    optimal_blend = {c: x[c].value for c in components}
    
    # composition (averages) of C, Cu, Mn
    avg_C = sum(x[c].value*data[c]['C'] for c in components) / mass 
    avg_Cu = sum(x[c].value*(data[c]['Cu']) for c in components) / mass
    avg_Mn = sum(x[c].value*(data[c]['Mn']) for c in components) / mass

    
    return min_cost, optimal_blend, avg_C, avg_Cu, avg_Mn

# define constant constraint variables
mass = 6500
carbon = [2.0, 3.0]
copper = [0.4, 0.6]
manganese = [1.2, 1.65]

# execute function and print results
min_cost, optimal_blend, avg_C, avg_Cu, avg_Mn = metal_blend(mass, carbon, copper, manganese, data)

print('With cost minimized:')
print(f"Mass: {mass} kg ")
print(f"Total cost: ${min_cost:5.2f}")
print(f"C composition: {avg_C:5.2f}%")
print(f"Cu composition: {avg_Cu:5.2f}%")
print(f"Mn composition: {avg_Mn:5.2f}%")
print()
for c in sorted(optimal_blend.keys()):
    print(f"{c}: {optimal_blend[c]:5.2f}", 'kg')
    
# cost per kg    
print()
print(f'Cost per kg: ${(min_cost/mass):5.2f}/kg')

With cost minimized:
Mass: 6500 kg 
Total cost: $8084.43
C composition:  2.00%
Cu composition:  0.60%
Mn composition:  1.20%

A: 4000.00 kg
B: 1000.00 kg
C: 80.74 kg
D:  0.00 kg
E: 34.60 kg
F: 1384.66 kg
G:  0.00 kg

Cost per kg: $ 1.24/kg


In [85]:
def metal_blend(mass, carbon, copper, manganese, data):
    
    # create lower and upper bounds from metal constraint inputs
    lower_boundC = carbon[0]
    upper_boundC = carbon[1]
    lower_boundCu = copper[0]
    upper_boundCu = copper[1]
    lower_boundMn = manganese[0]
    upper_boundMn = manganese[1]
    
    # create set of components
    components = set(data.keys())
    
    # create variables
    x = {c: cp.Variable(nonneg=True, name=c) for c in components}
    
    # create objective function
    total_cost = sum(x[c]*data[c]['cost'] for c in components)
    
    # create list of constraints
    constraints = [
        # mass
        mass == sum(x[c] for c in components),
        # carbon
        0 >= sum(x[c]*(data[c]['C'] - upper_boundC) for c in components),
        0 <= sum(x[c]*(data[c]['C'] - lower_boundC) for c in components),
        # copper
        0 >= sum(x[c]*(data[c]['Cu'] - upper_boundCu) for c in components),
        0 <= sum(x[c]*(data[c]['Cu'] - lower_boundCu) for c in components),
        # manganese
        0 >= sum(x[c]*(data[c]['Mn'] - upper_boundMn) for c in components),
        0 <= sum(x[c]*(data[c]['Mn'] - lower_boundMn) for c in components),
        # limits on each material
        0 >= x['A'] - (data['A']['amount']+1),
        0 >= x['B'] - data['B']['amount'],
        0 >= x['C'] - data['C']['amount'],
        0 >= x['D'] - data['D']['amount'],
        0 >= x['E'] - data['E']['amount'],
        0 >= x['F'] - data['F']['amount'],
        0 >= x['G'] - data['G']['amount']


    ]
    
    # create and solve problem
    problem = cp.Problem(cp.Minimize(total_cost), constraints)
    problem.solve()
    
    # return results
    # min cost and optimal blend
    min_cost = problem.value
    optimal_blend = {c: x[c].value for c in components}
    
    # composition (averages) of C, Cu, Mn
    avg_C = sum(x[c].value*data[c]['C'] for c in components) / mass 
    avg_Cu = sum(x[c].value*(data[c]['Cu']) for c in components) / mass
    avg_Mn = sum(x[c].value*(data[c]['Mn']) for c in components) / mass

    
    return min_cost, optimal_blend, avg_C, avg_Cu, avg_Mn

# define constant constraint variables
mass = 6500
carbon = [2.0, 3.0]
copper = [0.4, 0.6]
manganese = [1.2, 1.65]

# execute function and print results
min_cost, optimal_blend, avg_C, avg_Cu, avg_Mn = metal_blend(mass, carbon, copper, manganese, data)

print('With cost minimized:')
print(f"Mass: {mass} kg ")
print(f"Total cost: ${min_cost:5.2f}")
print(f"C composition: {avg_C:5.2f}%")
print(f"Cu composition: {avg_Cu:5.2f}%")
print(f"Mn composition: {avg_Mn:5.2f}%")
print()
for c in sorted(optimal_blend.keys()):
    print(f"{c}: {optimal_blend[c]:5.2f}", 'kg')
    
# cost per kg    
print()
print(f'Cost per kg: ${(min_cost/mass):5.2f}/kg')

With cost minimized:
Mass: 6500 kg 
Total cost: $8084.07
C composition:  2.00%
Cu composition:  0.60%
Mn composition:  1.20%

A: 4001.00 kg
B: 999.17 kg
C: 81.10 kg
D:  0.00 kg
E: 34.60 kg
F: 1384.12 kg
G:  0.00 kg

Cost per kg: $ 1.24/kg


I save 0.36 dollars in total given 1 additional kg of A. Thus, I would be willing to pay the supplier <= an additional 0.36 dollars/kg for additional A, or <= $1.56/kg for additional A.

<!--NAVIGATION-->
< [5.2 Linear Blending Problems](https://jckantor.github.io/cbe30338-2021/05.02-Linear-Blending-Problem.html) | [Contents](toc.html) | [Tag Index](tag_index.html) | [5.4 Gasoline Blending](https://jckantor.github.io/cbe30338-2021/05.04-Gasoline-Blending.html) ><p><a href="https://colab.research.google.com/github/jckantor/cbe30338-2021/blob/master/docs/05.03-Homework_5.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://jckantor.github.io/cbe30338-2021/05.03-Homework_5.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>